# Melify
At the moment, because it is rather complecated, we shall first take the test set and get the mean 
and variance. In this way we can proceed without conducting any preprocessing. The only change is that we will fix the normalization here and pass in the data for all trainings and testings. 

However, because at the moment I haven't found a good way to preserve good variance if using online methods, I will use the mean and variance as a whole, but from test set here to get all. 

In [ ]:
from model_dataset import TargetVowelDatasetManualNorm
from model_dataset import MelSpecTransformDBNoNorm as TheTransform
from paths import *
from model_dataset import Normalizer, DeNormalizer, TokenMap

import pandas as pd
import pickle

In [ ]:
REC_SAMPLE_RATE = 16000
N_FFT = 400
N_MELS = 64
LOADER_WORKER = 32

In [ ]:
t_set = pd.read_csv(os.path.join(src_, "phi-T-guide.csv"))
st_set = pd.read_csv(os.path.join(src_, "phi-ST-guide.csv"))
integrated = pd.concat([t_set, st_set], ignore_index=True, sort=False)
integrated = integrated.sample(frac=1).reset_index(drop=True)

In [ ]:
mytrans = TheTransform(sample_rate=REC_SAMPLE_RATE, 
                    n_fft=N_FFT, n_mels=N_MELS)

with open(os.path.join(src_, "no-stress-seg.dict"), "rb") as file:
    # Load the object from the file
    mylist = pickle.load(file)
    mylist = ["BLANK"] + mylist
    mylist = mylist + ["SIL"]

# Now you can use the loaded object
mymap = TokenMap(mylist)

mynorm = Normalizer(Normalizer.norm_mvn_manual)

In [ ]:
ds = TargetVowelDatasetManualNorm(
    src_dir=train_cut_phone_, guide_=integrated, 
    mapper=mymap, transform=mytrans, normalizer=mynorm, 
    noise_fixlength=False, noise_amplitude_scale=0.004, mv_config=None
)